<a href="https://colab.research.google.com/github/SpellOnYou/CLab21/blob/main/midterm/word2vec_window_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
from itertools import cycle

In [49]:
pip install -q ipdb

     |████████████████████████████████| 788kB 3.9MB/s 
     |████████████████████████████████| 368kB 8.1MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.23.1 which is incompatible.


In [50]:
from ipdb import set_trace

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [12]:
!cp -R /gdrive/MyDrive/2021/TL/2021-05-24/extracted/AA/. .

In [32]:
!head -n10 /content/processed/wiki_00

Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be undesirable, unnecessary, and harmful. As a historically far-left movement, it is usually described alongside libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement and has a strong historical association with anti-capitalism and socialism.
The history of anarchy goes back to prehistory, when humans arguably lived in anarchic societies long before the establishment of formal states, realms or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose, but it was not until the 19th century that a self-conscious political movement emerged. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a significant role in worke

In [2]:
src_path = Path('/content')
trg_path = Path('/content/processed')
trg_path.mkdir(parents = True, exist_ok = True)

In [3]:
def get_wiki(fname):
    if fname in list(i.name for i in src_path.iterdir()):
        all_sents = []
        with (trg_path/fname).open('w') as wf:
            with (src_path/fname).open() as rf:
                all_sents += [doc for doc in rf.read().split('\n\n') if doc if not doc.startswith('<doc') if not doc.startswith('</doc')]
            print('\n'.join(all_sents), file = wf)
        print(f"{fname} written done to {trg_path/fname}")

In [5]:
get_wiki('wiki_00')

wiki_00 written done to /content/processed/wiki_00


In [6]:
!cp /content/processed/wiki_00 /gdrive/MyDrive/2021/TL/2021-05-24/.

In [ ]:
trg_path

In [7]:
def get_data(wikiname):
    with (trg_path/wikiname).open('r') as f:
        ls = f.readlines()
    return ls

In [8]:
wiki_00=get_data('wiki_00')
len(wiki_00)

2751282

In [9]:
wiki_00_split = wiki_00[:int(len(wiki_00)*0.2)]

In [11]:
del wiki_00

In [10]:
len(wiki_00_split)

550256

# Vectorizer

In [16]:
class Vocabulary:
    def __init__(self, corpus):
        '''
        Args:
            corpus: List(str)
        '''    
        self.corpus = corpus
        self.sents = [list(map(self._cleansing_token, doc.split(' '))) for doc in corpus]
        self.vocab = list({token for sentence in self.sents for token in sentence}) + ['UNK']
        self._token_to_idx()
        self._idx_to_token()
        self.vocab_size = len(self._token_to_idx.keys())

    def _cleansing_token(self, token):
        return token.strip("\"\'\.\'\(\)\-").lower()
    def _token_to_idx(self):
        self._token_to_idx = {token:idx for idx, token in enumerate(self.vocab)}
    
    def _idx_to_token(self):
        self._idx_to_token = {idx:token for idx, token in enumerate(self.vocab)}

    def lookup_token(self, token):
        if token in self._token_to_idx:
            return self._token_to_idx[token]
        else:
            return self._token_to_idx['UNK']
        
    def lookup_idx(self, idx):
        """
        Args:
            idx: int
        """
        # assert type(idx) == int
        return self._idx_to_token[idx]

In [17]:
class Vectorizer(Vocabulary):
    """vectorizer which converts vocabulary tokens in sentence to word pairs given window size"""
    def __init__(self, *args):
        #inherit all method / attribute from vocab
        super().__init__(*args)
        # self.__call__(self)
        self.pairs = []

    def __call__(self, wd_size=2):
        for sent in self.sents:
            for idx, token in enumerate(sent):
                # print(sent)
                context_words = sent[(idx-wd_size):idx]+ sent[(idx+1): (idx+wd_size+1)]
                # print(sent, '\n', context_words, token); break
                context_indices = map(self.lookup_token, context_words)
                # print()
                self.pairs += [*zip(cycle([self.lookup_token(token)]), context_indices)]

In [18]:
vect = Vectorizer(wiki_00_split)
vect(2)

In [21]:
# del wiki_00_split
import gc
gc.collect()

550701

# Train embedding model

In [28]:
class Linear():
    def __init__(self, w, b):
        self.w, self.b = w, b
    def forward(self, x): 
        self.inp = x
        self.out = self.inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

class Relu():
    def forward(self, x):
        self.inp = x
        self.out = x.clamp_min(0.) - 0.5
        return self.out

    def backward(self):
        self.inp.g = self.out.g* (self.inp>0).float()        

class Softmax():
    def forward(self, x):
        self.inp = x
        self.out = (x.exp()/ x.exp().sum(-1, keepdim=True))
        return self.out

    def backward(self):
        self.inp.g = (1-self.out.g)*self.out.g

In [46]:
class CrossEntropy():
    def __call__(self, pred, y):
        
        self.yhat, self.y = pred, y
        #P(\hat{y})
        self.log_p_yhat = self.log_softmax(pred)
        self.out = self.nll(self.log_p_yhat, y)
        
        return self.out

    #negative log likelihood
    def nll(self, pred, y):
        # print(pred.shape, y.shape)
        return -pred[y]

    def log_softmax(self, x): return x - x.exp().sum(-1,keepdim=True).log()

    def backward(self, inp):
        set_trace()
        self.yhat.g = (inp.unsqueeze(1)*(self.yhat - self.y).unsqueeze(-1)).sum(-1)
        # self.yhat.g = (self.yhat.exp()/(self.yhat.exp().sum(-1,keepdim=True))) - self.y

class DummyModel():
    def __init__(self, w1, b1, w2, b2):
        self.loss = CrossEntropy()
        self.layers = [Linear(w1,b1), Relu(), Linear(w2, b2)]
    
    def forward(self, x):
        self.x= x

        for layer in self.layers:
            x = layer.forward(x)
        self.out = x
        return self.out

    def backward(self, x2):
        self.loss.backward(x2)
        for layer in reversed(self.layers):
            layer.backward()

In [33]:
from torch import randn

In [37]:
import torch, math

In [39]:
tv, emb_size = vect.vocab_size, 100

w1 = randn(v, emb_size) / math.sqrt(emb_size)
w2 = randn(emb_size, v) / math.sqrt(v)
b1 = randn(emb_size)
b2 = randn(v)

In [26]:
def get_x(x):
    inp = torch.zeros(v)
    inp[x] = 1
    return inp

In [55]:
model = DummyModel(w1, b1, w2, b2)
model.forward(get_x(vect.pairs[0][0])).shape

torch.Size([1014229])

In [ ]:

def train(epochs, lr):
    for e in range(epochs):
        for (x, y) in vect.pairs:
            tot_w_mean, tot_w_std = 0, 0
            # str_idx, end_idx = bs_i*bs, (bs_i+1)*bs
            # x_batch, y_batch = train_x[str_idx:end_idx], train_y[str_idx:end_idx]
            prediction = model.forward(get_x(x))
            loss = model.loss(prediction, y)

            model.backward(model.layers[-1].inp)
            
            with no_grad():
                for layer in model.layers:
                    if hasattr(layer, 'w'): #if they have parameter attribute
                        tot_w_mean+= layer.w.g.mean()
                        tot_w_std += layer.w.g.std()
                        layer.w -= layer.w.g * lr
                        layer.b -= layer.b.g * lr
                        layer.w.g.zero_() #initialize them to zero
                        layer.b.g.zero_()
            if bs_i % 20 ==0: print(tot_w_mean/bs, tot_w_std/bs)

In [52]:
train(1, 0.01)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



> <ipython-input-46-92837141e0e4>(20)backward()
     19         set_trace()
---> 20         self.yhat.g = (inp.unsqueeze(1)*(self.yhat - self.y).unsqueeze(-1)).sum(-1)
     21         # self.yhat.g = (self.yhat.exp()/(self.yhat.exp().sum(-1,keepdim=True))) - self.y

ipdb> inp.shape
torch.Size([100])
ipdb> self.yhat.shae
*** AttributeError: 'Tensor' object has no attribute 'shae'
ipdb> self.yhat.shape
torch.Size([1014229])
ipdb> self.y.shape
*** AttributeError: 'int' object has no attribute 'shape'
ipdb> self.y
1006682
--KeyboardInterrupt--
ipdb> q



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



BdbQuit: ignored

## Next: using vect.pairs,
- make one hot encoding with left side integer (X data)
- save right side integer to Y data